In [ ]:
File for conventional JPA parameters calculation - K,L,C,w_0.

In [1]:
from matplotlib import pyplot as plt
from numpy import linspace
import numpy as np
import scipy
import scipy.optimize

In [5]:
d = 2.8e-1 #length of resonator, cm
l =  4.23574*1e-9 #H/cm
#c = 1.04919*1e-12 #F/cm
c =  1.68135*1e-12 #F/cm
I_0 = 1.5e-6 #A
N = 1.
pi = 3.14159265359
hbar = 1.054e-34

fi_0 = 2.07e-15/2/pi
E_J = fi_0*I_0 / N
L_J = fi_0**2/E_J

k_0_0 = 0.5*pi/d
k_0 = k_0_0/(1 + L_J/(l*d)) #L_J<<ld

C = 0.5*c*d*( 1 + np.sin(2*k_0*d)/(2*k_0*d) )
L = 2*l*d/( (k_0*d)**2 * (1 + np.sin(2*k_0*d)/(2*k_0*d)) )  #Seems to be failed. Count it later.

w_0 = 1/(L*C)**0.5


fi_zpf = (hbar/(2*w_0*C))**0.5
K_i = -E_J/(2*hbar)*(fi_zpf*np.cos(k_0*d)/fi_0)**4/N**2 ###N**2 since EJ is modified

print (C*1e12, "C, pF")
print L*1e9, "L, nH"
print L_J*1e12, "L_J, pH"
print w_0/6.28e9, "f_res, GHz"
print K_i, "K"
print L_J/L*100, 'l_j/L, %'
print (l/c)**0.5, 'Z'
print 'Q >>', 8*L/L_J/1.71
print K_i/w_0

(0.2772504570173091, 'C, pF')
1.14647781426 L, nH
219.633821467 L_J, pH
8.9314381151 f_res, GHz
-7966470.00009 K
19.1572674791 l_j/L, %
50.1921250066 Z
Q >> 24.4208239938
-0.000142031570057


In [6]:
buf = 8*L/L_J/1.71
print 'Ck << ', (C/50./w_0/buf)**.5 *1e15 , 'fF'
print 1/(N * (8*6.28e9)**2 * L_J) * 1e12

Ck <<  63.6253900606 fF
1.80385456786


In [11]:
Ck = 8e-15
kappa = w_0**2*Ck**2* 50/(C+Ck)
gamma = .2 * kappa
Q = w_0/(gamma + kappa)*2
print 'kappa = ', kappa/6.28e6, 'MHz'
print 'Q =',Q
print Q/(8*L/L_J/1.71)
print 'P_crit = ', 10*np.log10( 4*1e3* hbar*w_0*(gamma + kappa)**3/ kappa/ np.abs(K_i)/27**.5 )
print 'P_therm =', 10*np.log10(hbar*w_0*kappa) + 30 + 20

kappa =  5.03722963104 MHz
Q = 2844.04551904
139.462513216
P_crit =  -122.422590698
P_therm = -107.447688095


In [27]:
print (4e-13/50*0.247e-12)**0.5*1e15

44.4522215418


In [267]:
h = 2000 #um
t = 40
b = 100e-6
s = 20
w = 100
epsilon = 3.8
N = 2.

A1 = 0.614*(h/s)**0.25*(t/h)**0.439
A2 = 0.775*t*(2*N - 1)/(t+s) + 0.408

print (epsilon + 1)*8.85e-12*b*(2*A1*(N-1) + A2 )*1e15 , 'pF'

8.317584 pF


In [111]:
h = 0.1 #um
t = 8.7
b = 20e-6
s = 5
w = 50
epsilon = 11.8
N = 2

A1 = 0.614*(h/s)**0.25*(t/h)**0.439
A2 = 0.775*t/(2*N - 1)/(t+s) + 0.408


print (epsilon + 1)*8.85e-12*b*(2*A1*(N-1) + A2 )

8.72773697084e-15


In [41]:
print 10 * np.log10(.5*hbar * w_0 * 1e3  *kappa*100)

-103.561000035


In [78]:
print hbar*5*6.28e12/fi_0

1.00456902247e-05


In [253]:
#Fingers
epsilon = 11.8 #substrate eps
W = 100e-6 #
s = 60e-6 #
g = 6e-6 #
h = 600e-6
epsilonEff = (epsilon + 1)/2 + (epsilon - 1)/2*( (1+ 12*h/W)**-0.5 + 0.04*(1 - W/h)**2  )
epsilon0 = 8.85e-12
p = g/W/4

C = 2*epsilon0*epsilonEff*W/3.1415*(p - (1 + p**2)**0.5 + np.log((1 + (1 + p**2)**0.5)/p))

print epsilonEff
print C *1e15

7.18202219486
15.8130203687


In [278]:
#Another empirical formula
l = 200
s = 5e-6
w = 5e-6
n = 10
epsilon = 11.8
C = 3.937e-5*l*(epsilon + 1)*(0.11*(n-3) + 0.252)

print C*1e3 , 'fF'

103.0045184 fF


In [1]:
#Analitical treatment for finger capacitor. Works.
w = 2e-6 #Electrod width
l = 4e-6 #Electrode length
g = 10e-6 #Space between electrodes
n = 2   #number of fingers

epsSub = 11.8  #epsilon of substrate
eps0 = 8.85e-12


Lambda = 2*(w+g)
eta = 2*w/Lambda
kE = w*eta**0.5/(1+ eta)
kI = np.sin(3.141596*eta/2)

def kMod(x):
    return((1 - x**2)*0.5)

def K(x):
    return(scipy.special.ellipk(x))

CI = eps0* l*(epsSub + 1)* K(kI)/K(kMod(kI))
CE = eps0* l*(epsSub + 1)* K(kE)/K(kMod(kE))

Ctot = (n-3)*0.5*CI + 2*CI*CE/(CI+CE)

print 'C =',Ctot*1e15, 'fF'
print (w+g)*n*1e6

NameError: name 'np' is not defined

In [24]:
print hbar * 6 * 6.28e9 / 1.38e-23

0.287787826087


## 